In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import re

In [2]:
#각 지역마다 필요한 열만 추출

path = './files/'
files = os.listdir(path)

cs = pd.DataFrame()

for file in files:
    data = pd.read_excel(path + file, usecols ='A,C,E')
    cs = pd.concat([cs,data])
    
cs_total = cs.reset_index(drop = True)

In [3]:
#결측값 확인
cs_total.isna().sum()

시도교육청    0
폐교년도     0
급별       0
dtype: int64

In [4]:
#nan 결측치 대체 (~초등 / ~중등 / ~고등)

for i in range(len(cs_total)):
    if cs_total['급별'].isna()[i] == True:
        
        if "초등" in cs_total['폐교명'][i]:
            cs_total['급별'].values[i] = '초'
            
        elif "중등" in cs_total['폐교명'][i] == True:
            cs_total['급별'].values[i] = '중'
            
        elif "고등" in cs_total['폐교명'][i] == True:
            cs_total['급별'].values[i] = '고'

In [5]:
#nan 결측치 대체 (~초 / ~중 / ~고)

for i in range(len(cs_total)):
    if cs_total['급별'].isna()[i] == True:
        
        str = cs_total["폐교명"][i]
        print(str)
        
        if str[-1] == "초":
            cs_total['급별'].values[i] = '초'
            
        elif str[-1] == '중':
            cs_total['급별'].values[i] = '중'
            
        elif str[-1] == '고':
            cs_total['급별'].values[i] = '고'

In [6]:
#필요 데이터만 추출
cs_close = (cs_total["폐교년도"] >= 2001) & (cs_total["폐교년도"] <= 2020)
cs_totals = cs_total[cs_close]

In [7]:
#폐교년도 타입 변경
cs_totals["폐교년도"] = cs_totals["폐교년도"].astype("str")

C:\Users\Doffal\AppData\Local\Temp/ipykernel_38212/809716553.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs_totals["폐교년도"] = cs_totals["폐교년도"].astype("str")


In [8]:
#인덱스 리셋
cs_totals = cs_totals.reset_index(drop =True)

In [9]:
#분석하기 위한 파일 저장
cs_totals.to_excel('./cs_totals.xlsx')

# 초 중 고등별 폐교데이터

In [10]:
time = ['2001', '2002', '2003', '2004', '2005', 
        '2006', '2007', '2008', '2009', '2010', 
        '2011', '2012', '2013', '2014', '2015', 
        '2016', '2017', '2018', '2019', '2020']

In [11]:
time_dic = {'폐교년도' : time}

In [12]:
#DataFrame형식 변환
time_df = pd.DataFrame(time_dic)

In [13]:
cs_emh = cs_totals.groupby(['폐교년도', '급별'], as_index =False).count()

In [14]:
ele = (cs_emh['급별'] == '초')
cs_ele = cs_emh[ele]

In [15]:
#left join
cs_ele_1 = pd.merge(time_df, cs_ele, how='left')
cs_ele_1 = cs_ele_1.fillna(0)

In [16]:
cs_ele_2 = cs_ele_1.drop(['폐교년도' ,'급별'], axis=1)

In [17]:
#중학교
mid = (cs_emh['급별'] == '중')
cs_mid = cs_emh[mid]

In [18]:
#left join
cs_mid_1 = pd.merge(time_df, cs_mid, how='left')
cs_mid_1 = cs_mid_1.fillna(0)

In [19]:
cs_mid_2 = cs_mid_1.drop(['폐교년도' ,'급별'], axis=1)

In [20]:
hig = (cs_emh['급별'] == '고')
cs_hig = cs_emh[hig]

In [21]:
#left join
cs_hig_1 = pd.merge(time_df, cs_hig, how='left')
cs_hig_1 = cs_hig_1.fillna(0)

In [22]:
cs_hig_2 = cs_hig_1.drop(['폐교년도' ,'급별'], axis=1)

In [23]:
cs_all_emh = pd.concat([time_df, cs_ele_2, cs_mid_2, cs_hig_2], axis=1)

In [24]:
cs_all_emh.columns = ['year', 'elementary', 'middle', 'high']

In [25]:
cs_all_emh['all'] = cs_all_emh['elementary'] + cs_all_emh['middle'] + cs_all_emh['high']